In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

from pyspark.mllib.classification import SVMModel, SVMWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors as MLLibVectors
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import * 
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder


from nltk.stem.porter import *
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

import pandas as pd
import string 
import re 

In [0]:
review = spark.read.load("s3://yelp-project-special-26/data-source/yelp_academic_dataset_review.json",format="json")

In [0]:
dbutils.fs.mkdirs("dbfs:/databricks/scripts/")

In [0]:
dbutils.fs.put("/databricks/scripts/nltk-install.sh",""" #!/bin/bash python -m pip install nltk python -m pip install --upgrade pip python -m nltk.downloader all """, True)

In [0]:
display(dbutils.fs.ls("/databricks/scripts/nltk-install.sh"))

In [0]:
!pip install s3fs

In [0]:
review.cache()

Out[5]: DataFrame[business_id: string, cool: bigint, date: string, funny: bigint, review_id: string, stars: double, text: string, useful: bigint, user_id: string]

In [0]:
# remove punctuation
def remove_punct(text):
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text) 
    return nopunct
    
# binarize rating
def convert_rating(rating):
    rating = int(rating)
    if rating < 2: return 1
    else: return 0

# udf
punct_remover = udf(lambda x: remove_punct(x))
rating_convert = udf(lambda x: convert_rating(x))
review_df = review.select('review_id', punct_remover('text'), rating_convert('stars'))
review_df = review_df.withColumnRenamed('<lambda>(text)', 'text')\
                     .withColumn('label', review_df["<lambda>(stars)"].cast(IntegerType()))\
                     .drop('<lambda>(stars)')\
                     .limit(5000)

In [0]:
review_df.show()

In [0]:
df=review_df.filter(review_df.label=='1')

In [0]:
#df.show(10)

In [0]:
df_badreviews=df.toPandas()

In [0]:
texts = []
for review in df_badreviews.text:
    texts.append(review)

In [0]:
texts

In [0]:
#Cleaning techniques #1 : Tokenization

In [0]:
import pandas as pd
import json
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

In [0]:
def tokenize(texts):
    tokenizer = nltk.RegexpTokenizer(r'\w+')

    texts_tokens = []
    for i, val in enumerate(texts):
        text_tokens = tokenizer.tokenize(val.lower())

        for i in range(len(text_tokens) - 1, -1, -1):
            if len(text_tokens[i]) < 4:
                del(text_tokens[i])

        texts_tokens.append(text_tokens)
        
    return texts_tokens

In [0]:
texts_tokens = tokenize(texts)

texts_tokens[:1]

Out[12]: [['this',
 'place',
 'used',
 'cool',
 'chill',
 'place',
 'bunch',
 'neanderthal',
 'bouncers',
 'hopped',
 'steroids',
 'acting',
 'like',
 'whatever',
 'they',
 'want',
 'there',
 'many',
 'better',
 'places',
 'davis',
 'square',
 'where',
 'they',
 'glad',
 'visiting',
 'their',
 'business',
 'that',
 'burren',
 'worst',
 'place',
 'davis']]

In [0]:
!pip install nltk

In [0]:
#Cleaning techniques #2 : Removing stopwords

In [0]:
def removeSW(texts_tokens):
    stopWords = set(stopwords.words('english'))
    texts_filtered = []

    for i, val in enumerate(texts_tokens):
        text_filtered = []
        for w in val:
            if w not in stopWords:
                text_filtered.append(w)
        texts_filtered.append(text_filtered)
        
    return texts_filtered

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!
Out[13]: True

In [0]:
texts_filtered = removeSW(texts_tokens)
texts_filtered[:1]

Out[15]: [['place',
 'used',
 'cool',
 'chill',
 'place',
 'bunch',
 'neanderthal',
 'bouncers',
 'hopped',
 'steroids',
 'acting',
 'like',
 'whatever',
 'want',
 'many',
 'better',
 'places',
 'davis',
 'square',
 'glad',
 'visiting',
 'business',
 'burren',
 'worst',
 'place',
 'davis']]

In [0]:
len(texts_filtered)

In [0]:
#Cleaning techniques #3 : Lemma

In [0]:
def lemma(texts_filtered):
    wordnet_lemmatizer = WordNetLemmatizer()
    texts_lem = []

    for i, val in enumerate(texts_filtered):
        text_lem = []
        for word in val:
            text_lem.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        texts_lem.append(text_lem)
    
    return texts_lem

In [0]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Package wordnet is already up-to-date!
Out[16]: True

In [0]:
texts_lem = lemma(texts_filtered)

texts_lem[:1]

Out[17]: [['place',
 'use',
 'cool',
 'chill',
 'place',
 'bunch',
 'neanderthal',
 'bouncers',
 'hop',
 'steroids',
 'act',
 'like',
 'whatever',
 'want',
 'many',
 'better',
 'place',
 'davis',
 'square',
 'glad',
 'visit',
 'business',
 'burren',
 'worst',
 'place',
 'davis']]

In [0]:
#BIGRAMS AND TRIGRAMS


In [0]:
nltk.download('wordnet')

In [0]:
len(texts_lem)

In [0]:
#Joinning

In [0]:
texts_string = []
for text in texts_lem:
    string = ' '
    string = string.join(text)
    texts_string.append(string)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3283707572726231> in <module> 
 1 texts_string = [ ] 
 ----> 2 for text in texts_lem : 
 3 string = ' ' 
 4 string = string . join ( text ) 
 5 texts_string . append ( string ) 

 NameError : name 'texts_lem' is not defined

In [0]:
nltk.download('stopwords')

In [0]:
texts_string[:10]

In [0]:
#LDA analysis with Sklearn

In [0]:
import pandas as pd
import json
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF

In [0]:
def plot_top_words(model, feature_names, n_top_words, title):
    #Modified from SKlearn
    fig, axes = plt.subplots(2, 5, figsize=(15, 7))
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f'Topic {topic_idx +1}',
                     fontdict={'fontsize': 15})
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=20)
        for i in 'top right left'.split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=15)
        ax.tick_params(bottom=False)
        ax.set(xticklabels=[])

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [0]:
vectorizer = CountVectorizer(max_df=0.90, min_df=5)


In [0]:
X = vectorizer.fit_transform(texts_string)
feature_names =  vectorizer.get_feature_names()

X.toarray().shape

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-63484592166280> in <module> 
 ----> 1 X = vectorizer . fit_transform ( texts_string ) 
 2 feature_names = vectorizer . get_feature_names ( ) 
 3 
 4 X . toarray ( ) . shape

 /databricks/python/lib/python3.7/site-packages/sklearn/feature_extraction/text.py in fit_transform (self, raw_documents, y) 
 1218 
 1219 vocabulary, X = self._count_vocab(raw_documents,
 -> 1220 self.fixed_vocabulary_)
 1221 
 1222 if self . binary : 

 /databricks/python/lib/python3.7/site-packages/sklearn/feature_extraction/text.py in _count_vocab (self, raw_documents, fixed_vocab) 
 1148 vocabulary = dict ( vocabulary ) 
 1149 if not vocabulary : 
 -> 1150 raise ValueError("empty vocabulary; perhaps the documents only"
 1151 " contain stop words")
 1152 

 ValueError : empty vocabulary; perhaps the documents only contain stop words

In [0]:
n_topics = 10

lda = LatentDirichletAllocation(
        n_components=n_topics,
        max_iter=12
)

lda.fit_transform(X)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-63484592166281> in <module> 
 6 )
 7 
 ----> 8 lda . fit_transform ( X ) 

 NameError : name 'X' is not defined

In [0]:
lda.n_iter_

In [0]:
#Displaying result of the analysis.

In [0]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic {}:".format(topic_idx))
        print(" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(lda, feature_names, no_top_words)

In [0]:
plot_top_words(lda, feature_names, no_top_words,'Topics in LDa')

In [0]:
#NMF analysis with Sklearn

In [0]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, 
    min_df=5,  
    stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(texts_string)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [0]:
from sklearn.decomposition import NMF


In [0]:
no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd')
nmf.fit_transform(tfidf)

In [0]:
no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)

In [0]:
plot_top_words(nmf, tfidf_feature_names, no_top_words,'Topics in NMF')

In [0]:
#LDA BoW and LDA TF-IDF with gensim

In [0]:

import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [0]:
!pip install pyLDAvis

In [0]:
!pip install -U gensim

In [0]:
#pip install --upgrade pip

In [0]:
dictionary = gensim.corpora.Dictionary(texts_lem)
len(dictionary.cfs)

In [0]:
dictionary.filter_extremes(no_below=15, no_above=0.5)
len(dictionary.cfs)

In [0]:
bow_corpus = [dictionary.doc2bow(doc) for doc in texts_lem]

In [0]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(bow_corpus))

In [0]:
tfidf = gensim.models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [0]:
#LDA using BoW

In [0]:
lda_model_coherence = []
for i in range (2,15):
    lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=i, id2word=dictionary, passes=2, workers=4)
    cm = gensim.models.CoherenceModel(model=lda_model, corpus=bow_corpus, coherence='u_mass')
    coherence = cm.get_coherence()
    lda_model_coherence.append(coherence)

In [0]:
plt.plot(range(2, 15),lda_model_coherence)
plt.xlabel('Number of topics')
plt.ylabel('Coherence score')
plt.title('How many topics ? (Closer to 0 = better)')
plt.show()

In [0]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=8, id2word=dictionary, passes=2, workers=4)

In [0]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [0]:
vis = gensimvis.prepare(topic_model=lda_model, corpus=bow_corpus, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [0]:
#LDA using TF-IDF

In [0]:
lda_model_tfidf_coherence = []
for i in range (2,15):
    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=i, id2word=dictionary, passes=2, workers=4)
    cm = gensim.models.CoherenceModel(model=lda_model_tfidf, corpus=bow_corpus, coherence='u_mass')
    coherence = cm.get_coherence()
    lda_model_tfidf_coherence.append(coherence)

In [0]:
plt.plot(range(2, 15),lda_model_coherence)
plt.xlabel('Number of topics')
plt.ylabel('Coherence score')
plt.title('How many topics ? (Closer to 0 = better)')
plt.show()

In [0]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=8, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [0]:
vis = gensimvis.prepare(topic_model=lda_model_tfidf, corpus=corpus_tfidf, dictionary=dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [0]:
#Scoring topics on documents

In [0]:
for i in range(0,3):
    print('### Scoring the document', i)
    for index, score in sorted(lda_model[bow_corpus[i]], key=lambda tup: -1*tup[1]):
        print("Score: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 5)), '\n')

In [0]:
for i in range(0,3):
    print('### Scoring the document', i)
    for index, score in sorted(lda_model_tfidf[bow_corpus[i]], key=lambda tup: -1*tup[1]):
        print("Score: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 5)), '\n')

In [0]:
#New reviews testing

In [0]:
def preprocess(raw_text):
    x = tokenize(raw_text)
    x = removeSW(x)
    x = lemma(x)
    return x

In [0]:
unseen_document = ['I had to wait 2 hours. It was so long. I will never come back here.', 'Food was horrible. My pizza was burn and was late.', 'It tasted bad. It is not good quality. Who is cooking here ?']
preprocessed_doc = preprocess(unseen_document)
bow_corpus = [dictionary.doc2bow(doc) for doc in preprocessed_doc]

print('There is', len(preprocessed_doc), 'reviews in the unseen document. We are going to calculate the score for the best topic of each of them.\n---------------\n')

for i in range(0, len(preprocessed_doc)):
    for index, score in sorted(lda_model[bow_corpus[i]], key=lambda tup: -1*tup[1]):
        print("# Review", i, ": best score: {}\t For topic: {}".format(score, lda_model.print_topic(index, 5)), '\n')
        break

In [0]:
nltk.download()

In [0]:
review.show(10)

In [0]:
df=review.toPandas()

In [0]:
from gensim import models

In [0]:
#BIGRAMS AND TRIGRAMS
bigrams_phrases=gensim.models.Phrases(texts_lem,min_count=5,threshold=100)
trigram_phrases=gensim.models.Phrases(bigrams_phrases[texts_lem],threshold=100)

bigram=gensim.models.phrases.Phraser(bigrams_phrases)
trigram=gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return(bigram[doc] for doc in texts)

def make_trigrams(texts):
    return(trigram[bigram[doc]] for doc in texts)

data_bigrams = make_bigrams(texts_lem)
data_bigrams_trigrams = make_trigrams(texts_lem)

#print(data_bigrams_trigrams)





In [0]:
data_bigrams_trigrams=list(data_bigrams_trigrams)
data_bigrams_trigrams



Out[22]: [['place',
 'use',
 'cool',
 'chill',
 'place',
 'bunch',
 'neanderthal',
 'bouncers',
 'hop',
 'steroids',
 'act',
 'like',
 'whatever',
 'want',
 'many',
 'better',
 'place',
 'davis',
 'square',
 'glad',
 'visit',
 'business',
 'burren',
 'worst',
 'place',
 'davis'],
 ['never',
 'seem',
 'order',
 'correct',
 'service',
 'crappy',
 'food',
 'inconsistent',
 'go',
 'hill',
 'steadily',
 'last',
 'months',
 'never'],
 ['wish',
 'could',
 'give',
 'zero',
 'star',
 'call',
 'takers',
 'rude',
 'technicians',
 'incompetent',
 'manager',
 'know',
 'customer',
 'service',
 'overprice',
 'quote',
 'go',
 'another',
 'locksmith',
 'actually',
 'lock',
 'quote',
 'total',
 'technician',
 'late',
 'twice',
 'could',
 'pick',
 'lock',
 'unusual',
 'say',
 'would',
 'drill',
 'lock',
 'fail',
 'bring',
 'charge',
 'batteries',
 'never',
 'offer',
 'come',
 'back',
 'charge',
 'batteries',
 'soon',
 'could',
 'offer',
 'send',
 'someone_else',
 'say',
 'could',
 'however',
 'come',
 'back',
 'around',
 'hours',
 'later',
 'table',
 'usually',
 'need',
 'lock',
 'need',
 'minute',
 'say',
 'charge',
 'service',
 'service',
 'never',
 'receive',
 'contact',
 'manager',
 'seem',
 'task',
 'say',
 'rightfully',
 'charge',
 'refuse',
 'technician',
 'offer',
 'come',
 'back',
 'later',
 'doubt',
 'manager',
 'know',
 'table',
 'payment',
 'still',
 'reimburse'],
 ['great',
 'coffee',
 'pastries',
 'baristas',
 'excellent',
 'staff',
 'sooooo',
 'lose',
 'vacant'],
 ['almost',
 'desolate',
 'restaurant',
 'dingy',
 'evironment',
 'star',
 'decieving',
 'lunch',
 'menu',
 'claim',
 'dish',
 'saturdays',
 'still',
 'charge',
 'star',
 'congeal',
 'beef',
 'soup',
 'star',
 'friend',
 'dish',
 'come',
 'minutes',
 'mine',
 'soup',
 'come',
 'star',
 'ask',
 'water',
 'time',
 'right',
 'bill',
 'come',
 'star',
 'welp',
 'seem',
 'star',
 'whoopsie',
 'daisy',
 'want',
 'forgive',
 'since',
 'waitress',
 'even',
 'manager',
 'step',
 'screw',
 'excuse',
 'mediocre',
 'food'],
 ['love',
 'house',
 'time',
 'star',
 'place',
 'service',
 'star',
 'management',
 'give',
 'absolutely',
 'zero',
 'respect',
 'game',
 'soccer',
 'come',
 'since',
 'open',
 'see',
 'numerous',
 'time',
 'understaff',
 'finals',
 'tournaments',
 'years',
 'past',
 'show',
 'capacity',
 'shame',
 'great',
 'establishment',
 'great',
 'employees',
 'management',
 'could',
 'sort',
 'place',
 'would',
 'star'],
 ['pleasant',
 'experience',
 'accord',
 'server',
 'restaurant',
 'short',
 'staff',
 'mean',
 'service',
 'frightfully',
 'slow',
 'hour',
 'receive',
 'food',
 'food',
 'good',
 'except',
 'steak',
 'tough',
 'trendy',
 'place',
 'great',
 'spot',
 'see',
 'food',
 'okay'],
 ['worst',
 'experience',
 'today',
 'try',
 'breakfast',
 'service',
 'slow',
 'wait',
 'least',
 'french',
 'toast',
 'pull',
 'long',
 'grosse',
 'hair',
 'never',
 'go'],
 ['bummer',
 'disappoint',
 'finally',
 'stop',
 'take',
 'fry',
 'haddock',
 'sandwich',
 'fry',
 'fry',
 'freeze',
 'greasy',
 'haddock',
 'sandwich',
 'good',
 'taste',
 'fishy',
 'freeze',
 'high',
 'hop',
 'take',
 'would',
 'good'],
 ['do',
 'lunch',
 'okay',
 'past',
 'today',
 'service',
 'failure',
 'multiple',
 'level',
 'first',
 'look',
 'long',
 'time',
 'food',
 'come',
 'waitress',
 'initially',
 'bring',
 'wrong',
 'order',
 'look',
 'minutes',
 'able',
 'chase',
 'waitress',
 'check',
 'finally',
 'receive',
 'check',
 'charge',
 'dinner',
 'price',
 'lunch',
 'run',
 'late',
 'back',
 'office',
 'pay',
 'say',
 'service',
 'inexcusable',
 'write',
 'hop',
 'take',
 'corrective',
 'action',
 'whether',
 'coach',
 'staff',
 'hire',
 'people',
 'lunch',
 'rush'],
 ['beer',
 'hardware',
 'problem',
 'least',
 'tap',
 'faulty',
 'sorry',
 'guy',
 'give',
 'years',
 'customers',
 'notice',
 'please',
 'scott'],
 ['roommate',
 'borrow',
 'clock',
 'radio',
 'apparently',
 'leave',
 'station',
 'mean',
 'wake',
 'normal',
 'radio',
 'alarm',
 'hear',
 'awful',
 'dude',
 'child',
 'apparently',
 'name',

In [0]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel
from gensim import corpora

id2word=corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus =[id2word.doc2bow(text) for text in texts]

print(corpus[0][0:20])

tfidf=TfidfModel(corpus,id2word=id2word)

low_value = 0.03

words = []

words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]         
    corpus[i] = new_bow


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 4), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]

In [0]:
# lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                            id2word=id2word,
#                                            num_topics=30,
#                                            random_state=100,
#                                            update_every=1,
#                                            chunksize=100,
#                                            passes=100,
#                                            alpha="auto")

In [0]:
lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=id2word, passes=100,random_state=100, workers=4,chunksize=100)

In [0]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.012*"place" + 0.009*"seat" + 0.008*"restaurant" + 0.007*"take" + 0.007*"receive" + 0.007*"good" + 0.006*"massage" + 0.006*"sell" + 0.006*"stand" + 0.006*"german"
Topic: 1 Word: 0.010*"make" + 0.010*"go" + 0.010*"call" + 0.009*"would" + 0.009*"tell" + 0.008*"clean" + 0.008*"back" + 0.006*"room" + 0.006*"take" + 0.006*"order"
Topic: 2 Word: 0.010*"come" + 0.008*"like" + 0.008*"place" + 0.007*"take" + 0.007*"order" + 0.007*"seem" + 0.006*"drink" + 0.006*"work" + 0.006*"coffee" + 0.006*"back"
Topic: 3 Word: 0.009*"tell" + 0.009*"time" + 0.008*"make" + 0.007*"room" + 0.006*"back" + 0.006*"place" + 0.006*"take" + 0.006*"much" + 0.005*"price" + 0.005*"call"
Topic: 4 Word: 0.010*"time" + 0.010*"place" + 0.010*"go" + 0.010*"door" + 0.009*"come" + 0.008*"wait" + 0.007*"disappoint" + 0.007*"service" + 0.007*"take" + 0.007*"never"
Topic: 5 Word: 0.016*"say" + 0.014*"service" + 0.014*"call" + 0.013*"back" + 0.010*"would" + 0.009*"tell" + 0.009*"ask" + 0.009*"customer" + 0.009*"order" + 0.008*"give"
Topic: 6 Word: 0.043*"food" + 0.022*"order" + 0.014*"table" + 0.014*"place" + 0.013*"wait" + 0.012*"service" + 0.011*"go" + 0.010*"good" + 0.009*"even" + 0.009*"minutes"
Topic: 7 Word: 0.012*"time" + 0.010*"would" + 0.009*"know" + 0.009*"like" + 0.008*"company" + 0.008*"leave" + 0.008*"say" + 0.007*"credit_card" + 0.007*"charge" + 0.006*"want"
Topic: 8 Word: 0.009*"phone" + 0.006*"even" + 0.006*"cookies" + 0.006*"like" + 0.006*"room" + 0.006*"take" + 0.005*"hard" + 0.005*"customer" + 0.005*"place" + 0.004*"charge"
Topic: 9 Word: 0.008*"make" + 0.007*"would" + 0.007*"ask" + 0.007*"come" + 0.006*"like" + 0.006*"food" + 0.006*"customers" + 0.006*"order" + 0.006*"manager" + 0.006*"give"

In [0]:
id2word.filter_extremes(no_below=15, no_above=0.5)
len(dictionary.cfs)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1530310699066947> in <module> 
 1 id2word . filter_extremes ( no_below = 15 , no_above = 0.5 ) 
 ----> 2 len ( dictionary . cfs ) 

 NameError : name 'dictionary' is not defined

In [0]:
vis = gensimvis.prepare(topic_model=lda_model, corpus=corpus, dictionary=id2word)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)

In [0]:
dictionary = gensim.corpora.Dictionary(corpus)


--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-1530310699066942> in <module> 
 ----> 1 dictionary = gensim . corpora . Dictionary ( corpus ) 

 /databricks/python/lib/python3.7/site-packages/gensim/corpora/dictionary.py in __init__ (self, documents, prune_at) 
 76 
 77 if documents is not None : 
 ---> 78 self . add_documents ( documents , prune_at = prune_at ) 
 79 self.add_lifecycle_event(
 80 "created" , 

 /databricks/python/lib/python3.7/site-packages/gensim/corpora/dictionary.py in add_documents (self, documents, prune_at) 
 200 
 201 # update Dictionary with the document 
 --> 202 self . doc2bow ( document , allow_update = True ) # ignore the result, here we only care about updating token ids 
 203 
 204 logger . info ( "built %s from %i documents (total %i corpus positions)" , self , self . num_docs , self . num_pos ) 

 /databricks/python/lib/python3.7/site-packages/gensim/corpora/dictionary.py in doc2bow (self, document, allow_update, return_missing) 
 242 counter = defaultdict ( int ) 
 243 for w in document : 
 --> 244 counter [ w if isinstance ( w , str ) else str ( w , 'utf-8' ) ] += 1 
 245 
 246 token2id = self . token2id

 TypeError : decoding to str: need a bytes-like object, tuple found

In [0]:
!pip install pyLDAvis

In [0]:
!pip uninstall pandas
!pip install pandas==1.1.5

In [0]:
#from pyLDAvis import gensim
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, id2word, mds='mmds' , R=30)
vis

In [0]:
vis = gensimvis.prepare(topic_model=lda_model, corpus=corpus,id2word=id2word,mds="mmds",R=30,dictionary)
pyLDAvis.enable_notebook()
pyLDAvis.display(vis)